In [ ]:
import pandas as pd
import os
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
# List of key locations with approximate lat/lon
locations = [
    {"NAME": "Bonneville Dam", "LAT": 45.643, "LON": -121.939, "TYPE": "Dam", "DAM": "BON"},
    {"NAME": "Ice Harbor Dam", "LAT": 46.2494, "LON": -118.8797, "TYPE": "Dam", "DAM": "IHR"},
    {"NAME": "John Day Dam", "LAT": 45.717, "LON": -120.709, "TYPE": "Dam", "DAM":"JDA"},
    {"NAME": "Little Goose Dam", "LAT": 46.5856, "LON": -118.0273, "TYPE": "Dam", "DAM":"LGS"},
    {
        "NAME": "Lower Granite Dam",
        "LAT": 46.6602,
        "LON": -117.4289,
        "TYPE": "Dam",  "DAM":"LGR"
    },
    {
        "NAME": "Lower Monument Dam",
        "LAT": 46.5618,
        "LON": -118.5383,
        "TYPE": "Dam", "DAM":"LMN"
    },
    {"NAME": "McNary Dam", "LAT": 45.9368, "LON": -119.2982, "TYPE": "Dam", "DAM":"MCN"},
    {
        "NAME": "Priest Rapids Dam",
        "LAT": 46.6440,
        "LON": -119.91189,
        "TYPE": "Dam", "DAM":"PRD"
    },
    {
        "NAME": "Rock Island",
        "LAT": 47.34249,
        "LON": -120.09343,
        "TYPE": "Dam", "DAM":"RIS"
    },
    {"NAME": "Rocky Reach Dam", "LAT": 47.53248, "LON": -120.2954, "TYPE": "Dam", "DAM":"RRH"},
    {"NAME": "The Dalles Dam", "LAT": 45.617, "LON": -121.130, "TYPE": "Dam", "DAM":"TDA"},
    {"NAME": "Wanapum Dam", "LAT": 46.8731, "LON": -119.9715, "TYPE": "Dam", "DAM":"WAN"},
    {
        "NAME": "Willamette Falls Dam",
        "LAT": 45.353456425637596,
        "LON": -122.6189,
        "TYPE": "Dam", "DAM":"WFA"
    },
    {"NAME": "Wells Dam", "LAT": 47.760, "LON": -119.9718, "TYPE": "Dam", "DAM":"WEL"},
]

dam_locations = pd.DataFrame(locations)
geometry = [Point(xy) for xy in zip(dam_locations["LON"], dam_locations["LAT"])]
gdf = gpd.GeoDataFrame(dam_locations, geometry=geometry, crs="EPSG:4326")

In [ ]:
output_dir = "../data/processed/GIS/important_locations"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
gdf.to_parquet(f"{output_dir}/columbia_snake_dams.parquet")

In [ ]:
f"{output_dir}/columbia_snake_dams.parquet"

In [ ]:
# Load marine areas
marine = gpd.read_parquet(
    "../data/processed/GIS/ocean/TERRITORIAL_WATERS.parquet"
).to_crs("EPSG:4326")
rivers = gpd.read_parquet(
    "../data/processed/GIS/inland_waters/US_INLAND_WATERS_COLUMBIA_R.parquet"
)
rivers = rivers.dissolve()

In [ ]:
# Create buffer at Columbia River mouth (12 NM = 22224 meters)
buffer_geom = (
    gpd.GeoSeries([Point(-123.95, 46.25)], crs="EPSG:4326")
    .to_crs(epsg=3857)
    .buffer(22224 * 2)
    .to_crs(epsg=4326)
)

# Clip marine zones to buffer
marine_clipped = marine[marine.intersects(buffer_geom.iloc[0])]
marine_clipped["geometry"] = marine_clipped.intersection(buffer_geom.iloc[0])

marine_clipped = marine_clipped.dissolve()

In [ ]:
marine_clipped["geometry"] = marine_clipped.difference(rivers)
marine_clipped = marine_clipped.explode()
marine_clipped = marine_clipped[marine_clipped.area == marine_clipped.area.max()]

marine_clipped.to_parquet(
    "../data/processed/GIS/ocean/TERRITORIAL_COLUMBIA_MOUNT.parquet"
)

In [ ]:
## Ocean Columbia Mouth
columbia_river_mouth = "../data/processed/GIS/ocean/TERRITORIAL_COLUMBIA_MOUNT.parquet"

## River Polygons
river_polygons = (
    "../data/processed/GIS/inland_waters/US_INLAND_WATERS_COLUMBIA_R.parquet"
)

## Important Areas Polygons
dam_points = "../data/processed/GIS/important_locations/columbia_snake_dams.parquet"

columbia_river_mouth = gpd.read_parquet(columbia_river_mouth).to_crs("EPSG:4326")
river_gdf = gpd.read_parquet(river_polygons).to_crs("EPSG:4326")
dams = gpd.read_parquet(dam_points).to_crs("EPSG:4326")

columbia_river_mouth = columbia_river_mouth.explode()
columbia_river_mouth["geometry"] = columbia_river_mouth["geometry"].buffer(0)

river_gdf = gpd.read_parquet(river_polygons).to_crs("EPSG:4326")
river_gdf = river_gdf.dissolve(by="RIVER_TYPE")
river_gdf = river_gdf.reset_index()[["RIVER_TYPE", "geometry"]]
river_gdf = river_gdf.explode()
river_gdf["geometry"] = river_gdf["geometry"].buffer(0)

river_gdf_MO = river_gdf[
    (river_gdf.RIVER_TYPE == "MAJOR") | (river_gdf.RIVER_TYPE == "OTHER")
]
river_gdf_MO["geometry"] = river_gdf_MO.geometry.force_2d()
river_gdf_MO = river_gdf_MO.dissolve()
river_gdf_MO["RIVER_TYPE"] = "MAJOR"

river_gdf_MI = river_gdf[
    ~((river_gdf.RIVER_TYPE == "MAJOR") | (river_gdf.RIVER_TYPE == "OTHER"))
]
river_gdf_MI["geometry"] = river_gdf_MI.geometry.force_2d()
river_gdf_MI = river_gdf_MI.dissolve()

river_gdf_MI["geometry"] = river_gdf_MI.difference(river_gdf_MO)

river_gdf = pd.concat([river_gdf_MI, river_gdf_MO])


dams = gpd.read_parquet(dam_points).to_crs("EPSG:4326")

In [ ]:
river_gdf.to_parquet(river_polygons)

In [ ]:
dams

In [ ]:
from shapely.geometry import mapping
import folium


def create_and_save_map(columbia_river_mouth, river_gdf, dams):
    # --- Color Map ---
    color_map = {
        "MAJOR": "#084594",
        "MINOR": "#deebf7",
    }

    # --- Folium Map ---
    m = folium.Map(location=[46.25, -123.95], zoom_start=8, tiles="CartoDB Positron")

    # --- Add River Polygons ---
    for _, row in river_gdf.iterrows():
        if row["geometry"] and not row["geometry"].is_empty:
            folium.GeoJson(
                mapping(row["geometry"]),
                name=f"{row['RIVER_TYPE']}",
                style_function=lambda x, row=row: {
                    "fillColor": color_map.get(row["RIVER_TYPE"], "#cccccc"),
                    "color": color_map.get(row["RIVER_TYPE"], "#cccccc"),
                    "weight": 1,
                    "fillOpacity": 0.6 if row["RIVER_TYPE"] != "MINOR" else 0.3,
                },
            ).add_to(m)

    # --- Add Marine Area Polygons ---
    for _, row in columbia_river_mouth.iterrows():
        if row["geometry"] and not row["geometry"].is_empty:
            folium.GeoJson(
                mapping(row["geometry"]),
                name="Marine Area",
                style_function=lambda x: {
                    "fillColor": "#a6bddb",
                    "color": "#3690c0",
                    "weight": 1,
                    "fillOpacity": 0.4,
                },
                tooltip="Marine Area",
            ).add_to(m)

    # --- Add Dams ---
    for _, row in dams.iterrows():
        if row.geometry.geom_type == "Point":
            folium.Marker(
                location=[row.geometry.y, row.geometry.x],
                fill_opacity=0.75,
                icon=folium.Icon(icon="fa-house-flood-water", prefix="fa"),
                color="#042A2B",
                fill_color="#042A2B",
                tooltip=row.get("NAME", "Dam"),
            ).add_to(m)

    # --- Add Layer Control ---
    folium.LayerControl().add_to(m)

    # --- Save ---
    # m.save("map.html")
    return m

In [ ]:
create_and_save_map(columbia_river_mouth, river_gdf, dams)

In [ ]:
# dams